In [1]:
%load_ext autoreload

In [2]:
%autoreload 
import logging
import os
import pathlib
import torch
from ssd.engine.inference import do_evaluation
from ssd.config.defaults import cfg
from ssd.utils.logger import setup_logger
from train import start_train

/opt/conda/lib/python3.6/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:541: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/opt/conda/lib/python3.6/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:542: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/opt/conda/lib/python3.6/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:543: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/opt/conda/lib/python3.6/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:544: FutureWa

In [3]:
#config_file = "configs/train_rdd2020_server.yaml"
#config_file = "configs/train_rdd2020_server_experimental_contrast.yaml"

config_file = "configs/train_rdd2020_server_experimental.yaml"

cfg.merge_from_file(config_file)
cfg.freeze()
output_dir = pathlib.Path(cfg.OUTPUT_DIR)
output_dir.mkdir(exist_ok=True, parents=True)

logger = setup_logger("SSD", output_dir)

logger.info("Loaded configuration file {}".format(config_file))
with open(config_file, "r") as cf:
    config_str = "\n" + cf.read()
    logger.info(config_str)
logger.info("Running with config:\n{}".format(cfg))


2021-04-23 17:44:09,265 SSD INFO: Loaded configuration file configs/train_rdd2020_server_experimental.yaml
2021-04-23 17:44:09,270 SSD INFO: 
MODEL:
    NUM_CLASSES: 5
    THRESHOLD: 0.5
    BACKBONE:
        NAME: 'res_net'
        PRETRAINED: True
        OUT_CHANNELS: [128, 256, 512, 512, 256, 128]
        INPUT_CHANNELS: 3
    PRIORS:
        MIN_SIZES:  [[4, 8], [21, 37], [43, 75], [64, 112], [106, 188], [170, 300]]
        MAX_SIZES:  [[21, 37], [43, 75], [64, 112], [106, 188], [170, 300], [268, 472]]
        FEATURE_MAPS:  [[32, 57], [16, 29], [8, 15], [4, 8], [2, 4], [1, 3]]
        STRIDES:  [[8, 8], [16, 17], [32, 30], [64, 65], [128, 113], [255, 225]]
        ASPECT_RATIOS: [[5.29], [3.53, 7.06], [3.53, 7.06], [3.53, 7.06], [7.06], [5.29]]

INPUT:
    IMAGE_SIZE: [255, 450]
DATASETS:
    TRAIN: ("rdd2020_train",)
    TEST: ("rdd2020_val", )
SOLVER:
    MAX_ITER: 120000
    GAMMA: 0.1
    BATCH_SIZE: 16
    LR: 1e-3
OUTPUT_DIR: 'outputs/rdd2020_255x450'
DATASET_DIR: "datasets

In [ ]:
!pwd 
model = start_train(cfg)

/notebooks/TDT4265-Project/assignment4/SSD
Detector initialized. Total Number of params:  21.46M
Backbone number of parameters: 20.67M
SSD Head number of parameters: 788.2K
SSDDetector(
  (backbone): ResNet18(
    (model): ResNet(
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): BasicBlock(
          (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (relu): ReLU(inplace=True)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_r

In [ ]:
logger.info('Start evaluating...')
torch.cuda.empty_cache()  # speed up evaluating after training finished
do_evaluation(cfg, model)